In [1]:
import ee

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project="co2calculator-407613")

In [80]:
#Load country data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Malaysia = countries.filter(ee.Filter.eq('country_na', 'Malaysia'))

In [113]:
dataset = ee.ImageCollection("FAO/GHG/1/DROSE_A")

In [114]:
croplandn2o = dataset.select('croplandn2o')\
.filterDate('1992-01-01', '2017-12-31')\
.filterBounds(Malaysia)\
.median()

grasslandc = dataset.select('grasslandc')\
.filterDate('2008-01-01', '2017-12-31')\
.filterBounds(Malaysia)\
.median()


In [121]:
croplandn2o_vis = {
  'bands': ['croplandn2o'],
  'min': 1,
  'max': 60,
  'palette': ['yellow']
}

grasslandc_vis = {
  'bands': ['grasslandc'],
  'min': 1,
  'max': 60,
  'palette': ['red']
}

In [122]:
import folium

In [123]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [124]:
# Create the map object.
my_map = folium.Map(location=[4.2105, 101.9758], zoom_start=6)

In [125]:
#Display
my_map.add_ee_layer(croplandn2o, croplandn2o_vis, 'Cropland NO2 emissions (Annual) YELLOW');
my_map.add_ee_layer(grasslandc, grasslandc_vis, 'Grassland C emissions (Annual) RED');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)